In [10]:
import pandas as pd
import numpy as np

from scipy.stats import linregress
import matplotlib.pyplot as plt

import datetime 

from colorama import Fore
from IPython.display import clear_output, display
from ipywidgets import IntSlider, Output
import ast 

import csv

In [11]:
def import_lexicon():
    # import the vocabulaire
    file = 'Lexicon.xlsx'
    df = pd.read_excel(file)
    return df

df_lexicon = import_lexicon()

In [12]:
def do_quiz1(no_of_q = 3):
    correct = 0
    false = 0
    previous_n = -1
    out = Output()
    words = Output()
    inp = Output()
    
    clear = Output()
    
    
    with words:
        print(df_lexicon['Polish'][correct])
        
    with clear:
        clear_output()

    while no_of_q > correct:
        print(no_of_q, correct)
        display(words)
        correct += 1
        display(clear)
        
        
        #user_input = input()
        #with out:
         #   user_input = input()
        #display(out)
          
        #if user_input == df_lexicon['Dutch'][correct]:
         #   correct += 1
          #  with out:
           #     clear_output()
            #display(out)

do_quiz1()

3 0


Output()

Output()

3 1


Output()

Output()

3 2


Output()

Output()

In [13]:
df_lexicon['Polish'][0]

'rzesień'

In [14]:
# for calculating your knowledge and ordering the words
def update_scores():
    
    # predict the number of correct answers based on the amount of tries
    # calculate slope and intercept (other vars are required, but not used)
    slope, intercept, r_value, p_value, std_err = linregress((df_lexicon['Correct'] + df_lexicon['False']), df_lexicon['Correct'])
    # calculate residuals based on slope and intercept
    df_lexicon['Residuals'] = df_lexicon['Correct'] - (slope * (df_lexicon['Correct'] + df_lexicon['False']) + intercept)

    # however, as the residuals of words with just a few tries will be low
    # eg (0correct,0false) has a low resid, (0,1) or (1,3) as well
    # I manually set the residual-value of words <= 8 tries (arbitrary border)
    # to -0.5, which as of now has them starting around position 30 (arbitrary position).
    df_lexicon.loc[((df_lexicon['Correct'] + df_lexicon['False']) <= 5), 'Residuals'] = -1.5

    # similarly, but to a more extreme extend, the words with fewer than 
    # 4 tries will get a residual of -1.5, starting them at #20 as of now
    df_lexicon.loc[((df_lexicon['Correct'] + df_lexicon['False']) <= 2), 'Residuals'] = -3
    
    # sort on residual and reset index
    df_lexicon.sort_values(by='Residuals', inplace=True, ascending=True)
    df_lexicon.reset_index(drop=True, inplace=True)
    
    # save results as xlsx
    writer = pd.ExcelWriter('Lexicon.xlsx')
    df_lexicon.to_excel(writer, 'Words')
    writer.save()
    # save results as csv
    df_lexicon.to_csv('Lexicon.csv', encoding='utf-8', index=False)

update_scores()

## also add: check when adding words whether they're already in the df

In [15]:
# for adding words 
def add_words(polish, dutch, english):
    
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    pd.to_datetime(date)


    df_lexicon.loc[len(df_lexicon)] = [polish, [dutch], [english], 0, 0, str(date), 0]
    print('Jeszcze raz?')
    answer = input()
    if answer == "n":
        writer = pd.ExcelWriter('Lexicon.xlsx')
        df_lexicon.to_excel(writer, 'Words')
        writer.save()
    else:
        a = answer
        b = input()
        c = input()
        add_words(a, b, c)
    clear_output()
    update_scores()

In [16]:
# "legacy" code
# works(-ish)

# MAAR
# je kunt niet extra opties toevoegen!!

def clear():
    clear_output(wait=True)

def do_quiz(no_of_q = 20):
    correct = 0
    false = 0
    false_queue = []
    previous_n = -1

    # quiz
    while no_of_q > correct:
        r = np.random.randint(1, 100)
        if r <= 50: # 60% chance to have one of the first 10% of words
            n = np.random.randint(0, (len(df_lexicon) * .15))                                
        elif r <= 95: # 35% chance for one of the words percentage 10 to 40
            n = np.random.randint(len(df_lexicon) * .15, len(df_lexicon) * .50) 
        else: # remaining 5% chance of the last 60% of words
            n = np.random.randint(20, len(df_lexicon))
        
        #clear_output()
        print(Fore.BLACK + df_lexicon['Polish'][n])
        #print(n, df_lexicon['Residuals'][n])
        #print(correct, false, no_of_q)
        
        # HERE:
        # print word color based on category it's from/residuals
        
        user_answer = input()        
        #print(user_answer)
        
        if user_answer == 'n':
            break
            
        elif user_answer == "lul nie":
            if previous_n != -1:
                df_lexicon.loc[previous_n, 'False'] -= 1
                df_lexicon.loc[previous_n, 'Correct'] += 1
                false_queue.remove(previous_n)
                # clear_output() clearing output removes the input()-prompt, 
                # even when specifically called again
                # maybe research why at a later moment
            
                print("Vooruit dan makker")
                user_answer = input("In what language?")
                
                if user_answer == "nl":
                    print("Toegevoegd aan antwoorden")
                    df_lexicon.loc[previous_n, 'Dutch'] = ast.literal_eval(df_lexicon.loc[previous_n, 'Dutch']).append(previous_answer)
                elif user_answer == "en":
                    print("Added to answers")
                    df_lexicon.loc[previous_n, 'English'] = ast.literal_eval(df_lexicon.loc[previous_n, 'English']).append(previous_answer)
    
                print(df_lexicon['Polish'][n])
                previous_n = -1
                correct += 1
                false -= 1
                user_answer = input()   
                           
        try:
            assert user_answer in ast.literal_eval(df_lexicon['Dutch'][n]) or user_answer in ast.literal_eval(df_lexicon['English'][n])
            correct += 1
            #clear_output()
            print(Fore.GREEN + "Correct!")
            df_lexicon.loc[n, 'Correct'] += 1
            #clear()
            #print(no_of_q, correct)
           
        except AssertionError:
            previous_n = n
            false += 1
            #clear_output()
            print(Fore.RED + "Your translation of '{0:5s}' for '{1:5s}' is not correct.".format(user_answer, df_lexicon['Polish'][n]))
            print(Fore.RED + "Try '{0:5s}' or '{1:5s}' next time!".format(ast.literal_eval(df_lexicon['English'][n])[0], ast.literal_eval(df_lexicon['Dutch'][n])[0]))
            df_lexicon.loc[n, 'False'] += 1
            false_queue.append(n)

            
        # progress
        if correct == (np.floor(no_of_q / 2)):
            print(Fore.GREEN + "Halfway there!")
        if correct == (np.floor(no_of_q * .75)):
            print(Fore.GREEN + "Almost there! Keep going!") 
        if correct == no_of_q:
            print(Fore.GREEN + "Finished with your original questions!")
            for n in false_queue:
                print(Fore.BLUE + df_lexicon['Polish'][n])

        previous_n = n
        previous_answer = user_answer
        
       
    # repetition of incorrect words
    while false_queue != []:
        for mistake in false_queue:
            print(false_queue)
            print(Fore.BLACK + "Only {0:3.0F} left to do!".format(len(false_queue)))
            print(Fore.BLACK + df_lexicon['Polish'][mistake])
            user_answer = input("Translation: ")
               
            if user_answer == "lul nie":
                if previous_n != -1:
                    df_lexicon.loc[previous_mistake, 'False'] -= 1
                    false_queue.remove(previous_mistake)
                    #clear_output()
                    print("Vooruit dan makker")
                    print(df_lexicon['Polish'][mistake])
                    previous_mistake = -1
                user_answer = input()
                
            try:
                assert user_answer in ast.literal_eval(df_lexicon['Dutch'][mistake]) or user_answer in ast.literal_eval(df_lexicon['English'][mistake])
                #clear_output()
                print(Fore.GREEN + "Correct!")
                false_queue.remove(mistake)
                
            except AssertionError:
                #clear_output()
                print(Fore.RED + "Your translation of '{0:5s}' for '{1:5s}' is not correct.".format(user_answer, df_lexicon['Polish'][mistake]))
                print(Fore.RED + "Try '{0:5s}' or '{1:5s}' next time!".format(ast.literal_eval(df_lexicon['English'][mistake])[0], ast.literal_eval(df_lexicon['Dutch'][mistake])[0])) 
                false_queue.append(mistake)
                df_lexicon.loc[mistake, 'False'] += 1
            
            previous_mistake = mistake
    
    print(Fore.BLACK + "Finished, great job!")
    print(Fore.BLACK + "Your final score is {0:3.1F}".format(float(10*correct/(correct+false))))
    
    update_scores()
    print(sum(df_lexicon['Residuals']))


In [17]:
do_quiz(15)

borówka
blauwe bes
Your translation of 'blauwe bes' for 'borówka' is not correct.
Try 'blueberry' or 'bosbes' next time!
pchać
lul nie
Vooruit dan makker
In what language?nee
pchać
duwen
Correct!
setny
honderdste
Correct!
kto to jest?
wie is dat?
Correct!
styczeń
januari
Correct!
kobieta
vrouw
Correct!
dla palących
roken
Correct!
Halfway there!
w środku
's middags
Your translation of ''s middags' for 'w środku' is not correct.
Try 'in the middle' or 'in het midden' next time!
Halfway there!
płeć
duwen
Your translation of 'duwen' for 'płeć ' is not correct.
Try 'sex  ' or 'geslacht' next time!
Halfway there!
oczywiście
natuurlijk
Correct!
styczeń
januari
Correct!
korzystać
brengen
Your translation of 'brengen' for 'korzystać' is not correct.
Try 'to use' or 'gebruiken' next time!
mężczyzna
man
Correct!
dziewięćdziesiąt
negentig
Correct!
Almost there! Keep going!
gorąco
heet
Correct!
jak pan (pani) się nazywa?
hoe heet u?
Correct!
bagaż
baggage
Correct!
przychodzić
vertrekken
Your transl

In [84]:
df_copy = df_lexicon.copy()
print("Toegevoegd aan antwoorden")
temp = df_copy.loc[1, 'Dutch']
temp 
#= temp.append('ook al')

#df_copy.head(5)             

Toegevoegd aan antwoorden


"['oktober']"

In [87]:
print([x for x in (df_copy.loc[3, 'Dutch'])])
      

#      , 'ook goed'])
temp = df_copy.loc[2, 'Dutch']
temp.append('ok')

df_copy.at[3, 'Dutch'] = [df_copy.loc[3, 'Dutch'], 'ook goed']

["['geduldig']", 'ook goed']


AttributeError: 'str' object has no attribute 'append'

In [82]:
temp.append("oa")
temp

AttributeError: 'NoneType' object has no attribute 'append'

In [57]:
df_copy.head(5)

,Polish,Dutch,English,Correct,False,Date added,Residuals
0,rzesień,None,['September'],15,33,2018-09-08,-10.962776
1,październik,['oktober'],['October'],11,28,2018-09-08,-10.602803
2,po,[['na']],['after'],17,32,2018-08-31,-9.447217
3,cierpliwy,"[[[[, ook goed], ook goed], ook goed]",['patient'],9,19,2018-09-11,-7.273947
4,odprowadzić,['brengen'],['to bring'],4,11,2018-09-12,-5.976209


In [89]:
pools_t = [['[a]'], ['[b]'], ['[c]']]
nl_t = ['1', '2', '3']
df_temp = pd.DataFrame({'pools_t': pools_t, 'nl_t': nl_t})
df_temp

,nl_t,pools_t
0,1,[[a]]
1,2,[[b]]
2,3,[[c]]


In [90]:
b = df_temp.loc[0, 'pools_t']
b.append('ok')
b



['[a]', 'ok']

In [6]:
def clear():
    clear_output(wait=True)

def do_quiz1(no_of_q = 20):
    correct = 0
    false = 0
    previous_n = -1

    while no_of_q > correct:
        n = np.random.randint(0, 10)                                
        print(df_lexicon['Polish'][n])
        print(no_of_q, correct, false)
        user_answer = input()         

                                       
        try:
            assert user_answer in ast.literal_eval(df_lexicon['Dutch'][n]) or user_answer in ast.literal_eval(df_lexicon['English'][n])
            correct += 1
            print(Fore.GREEN + "Correct!")
            clear()
            print(no_of_q, correct)
           
        except AssertionError:
            previous_n = n
            false += 1
            print(Fore.RED + "Your translation of '{0:5s}' for '{1:5s}' is not correct.".format(user_answer, df_lexicon['Polish'][n]))
            print(Fore.RED + "Try '{0:5s}' or '{1:5s}' next time!".format(ast.literal_eval(df_lexicon['English'][n])[0], ast.literal_eval(df_lexicon['Dutch'][n])[0]))

        previous_n = n
        previous_answer = user_answer
 

In [ ]:
do_quiz1()

In [ ]:
a = 2

In [78]:
df_lexicon.loc[520:, 'Dutch']

520        ['datief']
521    ['dit', 'dat']
522       ['dochter']
523      ['genitief']
524          ['dame']
525     ['enkelvoud']
526         ['ouder']
Name: Dutch, dtype: object

In [79]:
df_lexicon.loc[521]

Polish                         to
Dutch              ['dit', 'dat']
English          ['this', 'that']
Correct                        13
False                           0
Date added    2018-08-22 00:00:00
Residuals                 4.06032
Name: 521, dtype: object

In [ ]:
antwoord = 'dat'
assert antwoord in df_lexicon['Dutch'][521] or antwoord in df_lexicon['English'][521]
print(Fore.RED + "Try '{0:5s}' or '{1:5s}' next time!".format(df_lexicon['English'][521], df_lexicon['Dutch'][521]))   

In [87]:
assert 't' in ast.literal_eval(df_lexicon['English'][521])

AssertionError: 

In [13]:
sum(df_lexicon['Residuals']), 100* sum(df_lexicon['Correct'])/(sum(df_lexicon['Correct']) + sum(df_lexicon['False'])),  sum(df_lexicon['Correct']) + sum(df_lexicon['False']), 

(-103.76088247173429, 77.14523166424267, 4813)

In [183]:
sum(df_lexicon['Residuals']), 100*sum(df_lexicon['Correct'])/(sum(df_lexicon['Correct']) + sum(df_lexicon['False'])),  sum(df_lexicon['Correct']) + sum(df_lexicon['False']), 

(-118.97630833371049, 77.16617831728757, 4778)

In [181]:
# To see the words with < 3 tries
df_lexicon[:26]

,Polish,Dutch,English,Correct,False,Date added,Residuals
0,rzesień,[september],[September],13,33,2018-09-08 00:00:00,-12.225789
1,październik,[oktober],[October],11,28,2018-09-08 00:00:00,-10.769843
2,po,[na],[after],15,31,2018-08-31 00:00:00,-10.225789
3,cierpliwy,[geduldig],[patient],6,19,2018-09-11 00:00:00,-8.857950
4,styczeń,[januari],[January],18,24,2018-09-08 00:00:00,-5.250963
5,zacząć,[beginnen],[to begin],4,9,2018-09-12 00:00:00,-4.933470
6,odprowadzić,[brengen],[to bring],4,9,2018-09-12 00:00:00,-4.933470
7,kwiecień,[april],[April],22,26,2018-09-08 00:00:00,-4.213203
8,czerwiec,[juni],[June],4,7,2018-09-08 00:00:00,-3.946057
9,brzydki,[lelijk],[ugly],7,10,2018-09-09 00:00:00,-3.908296


In [182]:
# To see the words with < 6 tries
df_lexicon[20:165]

,Polish,Dutch,English,Correct,False,Date added,Residuals
20,powiedzieć,[vertellen],[to tell],1,0,2019-03-26,-3.000000
21,musieć,[moeten],[to have to],1,0,2019-03-26,-3.000000
22,wszystek,[alle],[all the],0,0,2019-03-26,-3.000000
23,sam,[alleen],[alone],3,3,2018-09-12 00:00:00,-2.477523
24,korzystać,[gebruiken],[to use],5,5,2018-09-09 00:00:00,-2.452350
25,czarny,[zwart],[black],4,3,2018-09-11 00:00:00,-1.971230
26,móc,[kunnen],[being able],4,3,2018-09-09 00:00:00,-1.971230
27,panie,[dames],[ladies],4,3,2018-08-24 00:00:00,-1.971230
28,przedwczoraj,[eergisteren],[the day before yesterday],5,4,2018-09-06 00:00:00,-1.958643
29,dom,[huis],[house],5,0,2018-08-27 00:00:00,-1.500000


In [81]:
df_lexicon.Residuals.value_counts()

-1.500000     132
 0.549485      99
 1.053197      30
-0.450515      23
 0.053197      19
-3.000000      13
 1.556909      12
-0.443091      11
-0.946803      10
-1.450515       8
 0.556909       7
 3.571755       7
-0.428245       6
-0.398553       5
 0.068043       5
 0.060620       5
 1.060620       4
-0.413399       4
 0.090313       4
-0.435668       4
-1.428245       4
 0.082890       4
 0.579178       4
-0.420822       4
 2.060620       3
 1.075466       3
-1.435668       3
 2.068043       3
 0.105159       3
-1.443091       2
             ... 
 2.112582       1
 0.157121       1
 0.594024       1
 1.616294       1
-5.435668       1
 2.571755       1
 0.564332       1
-1.924534       1
 3.631140       1
 4.594024       1
-2.435668       1
 3.645986       1
-10.828033      1
 2.564332       1
 7.638563       1
 1.186814       1
-4.924534       1
-0.909687       1
 1.068043       1
-5.316898       1
 2.594024       1
-0.405976       1
-3.413399       1
-0.368860       1
 3.683102 

In [129]:
for i in range(len(df_lexicon)):
    f = {df_lexicon.loc[i, 'English']}
    print(f)
    df_lexicon.loc[i, 'English'] = f
    
    g = {df_lexicon.loc[i, 'Dutch']}
    print(g)
    df_lexicon.loc[i, 'Dutch'] = g
df_lexicon

{'September'}
{'september'}
{'October'}
{'oktober'}
{'after'}
{'na'}
{'patient'}
{'geduldig'}
{'to begin'}
{'beginnen'}
{'January'}
{'januari'}
{'to bring'}
{'brengen'}
{'April'}
{'april'}
{'June'}
{'juni'}
{'ugly'}
{'lelijk'}
{'of course'}
{'natuurlijk'}
{'to have to'}
{'moeten'}
{'to know'}
{'weten'}
{'thing'}
{'ding'}
{'to tell'}
{'vertellen'}
{'so'}
{'dus'}
{'all the'}
{'alle'}
{'to speak'}
{'spreken'}
{'to have'}
{'hebben'}
{'to stay'}
{'blijven'}
{'however'}
{'echter'}
{'year'}
{'jaar'}
{'banana'}
{'banaan'}
{'and'}
{'en'}
{'alone'}
{'alleen'}
{'to use'}
{'gebruiken'}
{'being able'}
{'kunnen'}
{'black'}
{'zwart'}
{'ladies'}
{'dames'}
{'the day before yesterday'}
{'eergisteren'}
{'even'}
{'zelfs'}
{'one'}
{'een'}
{'thursday'}
{'donderdag'}
{'lettuce'}
{'sla'}
{'strawberry'}
{'aardbei'}
{'cherry'}
{'kers'}
{'blackberry'}
{'braam'}
{'coconut'}
{'kokosnoot'}
{'plum'}
{'pruim'}
{'eighty'}
{'tachtig'}
{'hundredth'}
{'honderdste'}
{'bureau'}
{'bureau'}
{'light'}
{'licht'}
{'talented'}
{

{'to laugh'}
{'lachen'}
{'six'}
{'zes'}
{'watch out'}
{'let op'}
{'monday'}
{'maandag'}
{'cheese'}
{'kaas'}
{'pool'}
{'zwembad'}
{'one'}
{'een'}
{'large'}
{'groot'}
{'nine hundred'}
{'negenhonderd'}
{'good'}
{'goed'}
{'to pay'}
{'betalen'}
{'you'}
{'jij'}
{'new'}
{'nieuw'}
{'going straight'}
{'rechtdoor gaan'}
{'seventeen'}
{'zeventien'}
{'what a nonsense'}
{'wat een onzin'}
{'kiwi'}
{'kiwi'}
{'hot'}
{'heet'}
{'May'}
{'mei'}
{'carrot'}
{'wortel'}
{'drink'}
{'drankje'}
{'wine'}
{'wijn'}
{'a bit'}
{'een beetje'}
{'hour'}
{'uur'}
{'boy'}
{'jongen'}
{'they drink'}
{'zij drinken'}
{'thirty'}
{'dertig'}
{'whether'}
{'of'}
{'breakfast'}
{'ontbijt'}
{'reception'}
{'receptie'}
{'here'}
{'hier'}
{'closed'}
{'gesloten'}
{'I cook'}
{'ik kook'}
{'twenty'}
{'twintig'}
{'first aid'}
{'eerste hulp'}
{'male'}
{'mannelijk'}
{"don't you like dancing?"}
{'houd je niet van dansen?'}
{'what'}
{'wat'}
{'too late'}
{'te laat'}
{'tickets'}
{'kaartjes'}
{'room for rent'}
{'kamer te huur'}
{'third'}
{'derde'}
{'

,Polish,Dutch,English,Correct,False,Date added,Residuals
0,rzesień,{september},{September},13,33,2018-09-08 00:00:00,-12.239206
1,październik,{oktober},{October},11,28,2018-09-08 00:00:00,-10.780347
2,po,{na},{after},15,31,2018-08-31 00:00:00,-10.239206
3,cierpliwy,{geduldig},{patient},6,19,2018-09-11 00:00:00,-8.862627
4,zacząć,{beginnen},{to begin},3,9,2018-09-12 00:00:00,-5.439030
5,styczeń,{januari},{January},18,24,2018-09-08 00:00:00,-5.262715
6,odprowadzić,{brengen},{to bring},4,9,2018-09-12 00:00:00,-4.933153
7,kwiecień,{april},{April},22,26,2018-09-08 00:00:00,-4.227452
8,czerwiec,{juni},{June},4,7,2018-09-08 00:00:00,-3.944908
9,brzydki,{lelijk},{ugly},7,10,2018-09-09 00:00:00,-3.909645


In [164]:
for i in range(len(df_lexicon)):
    k = df_lexicon.loc[i, 'Dutch'].pop()
    print(k)
    l = [k]
    df_lexicon.loc[i, 'Dutch'] = l

september
oktober
na
geduldig
januari
beginnen
brengen
april
juni
lelijk
natuurlijk
moeten
weten
ding
vertellen
dus
jaar
alle
hebben
blijven
echter
spreken
banaan
en
alleen
gebruiken
zwart
dames
kunnen
eergisteren
braam
honderdste
tachtig
pruim
kokosnoot
kers
amandel
sla
donderdag
een
zelfs
bureau
aardbei
licht
toiletten
acht
achtste
augustus
paddestoel
romantisch
hoe laat?
pistache
dertiende
nul
willen
bosbes
honingmeloen
ja
jullie drinken
getalenteerd
rijst
kip
warme gerechten
oranje
water
tiende
ik heb haast
thee
maand
slechts
de helft
achttien
tweeentwintigste
duizend
rood
zeven
feest
uitgang
zonder
december
gesloten
kalender
arm
druif
boter
wonen
dit
minuut
rechts
vijftiende
aardappel
framboos
het regent
been
roze
rode biet
vrouwelijk
elf
wortel
manager
ei
zevenhonderd
bacon
zeshonderd
brief
eenentwintigste
reizen
walnoot
groen
honderdeneen
te vroeg
fruit
egoïst
knoflook
broccoli
gezond
taart
koud
warme chocolademelk
tweeduizend
tot ziens
twaalf
open
cola
huis
blauw
geel
lach
hij 

In [74]:
df_lexicon

,Polish,Dutch,English,Correct,False,Date added,Residuals
0,rzesień,['september'],['September'],13,33,2018-09-08,-12.236006
1,październik,['oktober'],['October'],11,28,2018-09-08,-10.779210
2,po,['na'],['after'],15,31,2018-08-31,-10.236006
3,cierpliwy,['geduldig'],['patient'],6,19,2018-09-11,-8.865617
4,odprowadzić,['brengen'],['to bring'],4,10,2018-09-12,-5.433509
5,styczeń,['januari'],['January'],18,24,2018-09-08,-5.260694
6,zacząć,['beginnen'],['to begin'],4,9,2018-09-12,-4.939681
7,kwiecień,['april'],['April'],22,26,2018-09-08,-4.223662
8,czerwiec,['juni'],['June'],4,7,2018-09-08,-3.952025
9,brzydki,['lelijk'],['ugly'],7,10,2018-09-09,-3.914993


In [70]:
add_words(input(), input(), input())

In [32]:
d = {'col4': [['airconditioning', 'air conditioning'], [2]], 'col2': [3, 4], 'col3': ['janjaap', 'freque']}
df_test = pd.DataFrame(data=d)
df_test

,col2,col3,col4
0,3,janjaap,"[airconditioning, air conditioning]"
1,4,freque,[2]


In [73]:
df_test['col4'][0]

['airconditioning', 'air conditioning']

In [37]:
n = 'airco'

assert n in df_test['col4'][0]


AssertionError: 

In [34]:
df_test['col4'][0]

['airconditioning', 'air conditioning']

In [707]:
file_list = ['C:/Users/Rodger/Documents/Data Science/[Portfolio]/Learning Polish/pl196x/a-publi.xml',
             ]

In [834]:
# Convert xml to string
# this is required since two tags near the end of the file throw errors
# so they will have to be removed


df_ = pd.DataFrame()

for path in file_list:
    xml_string = ""
    f = open(path, encoding='utf-8')

    for line in f:
        line = line.strip()
        xml_string += line

    xml_string = xml_string[:7755222] + xml_string[7755235:]

    element = ET.fromstring(xml_string)
    tree = ET.ElementTree(element)
    root = tree.getroot()
    print("Just finished: " + path)
    

    all_text = [(txt.tag) for txt in root.iter('text')]
    #all_words = [(elem.text, elem.get('lemma'), elem.tag) for elem in root.iter()]

    #df_ = df_.append(all_words)

#df_.columns = ['word', 'lemma', 'id']

Just finished: C:/Users/Rodger/Documents/Data Science/[Portfolio]/Learning Polish/pl196x/a-publi.xml
teiHeader {'type': 'text', 'status': 'update', 'creator': 'Maciej Ogrodniczuk', 'date.created': '2000-09-20', 'date.updated': '2004-06-16'}
text {'lang': 'pl'}


In [835]:
all_text

[{'lang': 'pl'},
 {'id': 'pu0001'},
 {'id': 'pu0002'},
 {'id': 'pu0003'},
 {'id': 'pu0004'},
 {'id': 'pu0005'},
 {'id': 'pu0006'},
 {'id': 'pu0007'},
 {'id': 'pu0008'},
 {'id': 'pu0009'},
 {'id': 'pu0010'},
 {'id': 'pu0011'},
 {'id': 'pu0012'},
 {'id': 'pu0013'},
 {'id': 'pu0014'},
 {'id': 'pu0015'},
 {'id': 'pu0016'},
 {'id': 'pu0017'},
 {'id': 'pu0018'},
 {'id': 'pu0019'},
 {'id': 'pu0020'},
 {'id': 'pu0021'},
 {'id': 'pu0022'},
 {'id': 'pu0023'},
 {'id': 'pu0024'},
 {'id': 'pu0025'},
 {'id': 'pu0026'},
 {'id': 'pu0027'},
 {'id': 'pu0028'},
 {'id': 'pu0029'},
 {'id': 'pu0030'},
 {'id': 'pu0031'},
 {'id': 'pu0032'},
 {'id': 'pu0033'},
 {'id': 'pu0034'},
 {'id': 'pu0035'},
 {'id': 'pu0036'},
 {'id': 'pu0037'},
 {'id': 'pu0038'},
 {'id': 'pu0039'},
 {'id': 'pu0040'},
 {'id': 'pu0041'},
 {'id': 'pu0042'},
 {'id': 'pu0043'},
 {'id': 'pu0044'},
 {'id': 'pu0045'},
 {'id': 'pu0046'},
 {'id': 'pu0047'},
 {'id': 'pu0048'},
 {'id': 'pu0049'},
 {'id': 'pu0050'},
 {'id': 'pu0051'},
 {'id': 'pu005

In [836]:
raw_txt[0:500]

['<?xml version="1.0" encoding="utf-8"?>',
 '',
 '',
 '<TEI.2 id="stylA">',
 '',
 '<!-- File included by pl196x.xml. -->',
 '',
 '<teiHeader type="text" status="update" creator="Maciej Ogrodniczuk" date.created="2000-09-20" date.updated="2004-06-16">',
 '<fileDesc>',
 '<titleStmt>',
 '<title lang="pl">Polszczyzna lat sześćdziesiątych. Wersja elektroniczna stylu A: publicystyka.</title>',
 '<title lang="en">Polish language of the 1960s. Electronic version of style A: essays.</title>',
 '',
 '<respStmt>',
 '<resp lang="pl">Twórcy korpusu na potrzeby Słownika Frekwencyjnego Polszczyzny Współczesnej</resp>',
 '<resp lang="en">Creators of the corpus for Frequency Dictionary of Contemporary Polish</resp>',
 '<name>Ida Kurcz</name>',
 '<name>Jadwiga Sambor</name>',
 '<name>Andrzej Lewicki</name>',
 '<name>Jerzy Woronczak</name>',
 '<name>Krzysztof Szafran</name>',
 '</respStmt>',
 '</titleStmt>',
 '',
 '<publicationStmt>',
 '<distributor lang="pl">Katedra Lingwistyki Formalnej, Uniwersytet Wa

In [871]:
raw_txt_2 = raw_txt[73:148]
element = ET.fromstring(raw_txt_2)
tree = ET.ElementTree(element)
root = tree.getroot()

TypeError: a bytes-like object is required, not 'list'

In [869]:
all_text = [(txt.attrib) for txt in root.iter('text')]

In [870]:
all_text

[{'lang': 'pl'},
 {'id': 'pu0001'},
 {'id': 'pu0002'},
 {'id': 'pu0003'},
 {'id': 'pu0004'},
 {'id': 'pu0005'},
 {'id': 'pu0006'},
 {'id': 'pu0007'},
 {'id': 'pu0008'},
 {'id': 'pu0009'},
 {'id': 'pu0010'},
 {'id': 'pu0011'},
 {'id': 'pu0012'},
 {'id': 'pu0013'},
 {'id': 'pu0014'},
 {'id': 'pu0015'},
 {'id': 'pu0016'},
 {'id': 'pu0017'},
 {'id': 'pu0018'},
 {'id': 'pu0019'},
 {'id': 'pu0020'},
 {'id': 'pu0021'},
 {'id': 'pu0022'},
 {'id': 'pu0023'},
 {'id': 'pu0024'},
 {'id': 'pu0025'},
 {'id': 'pu0026'},
 {'id': 'pu0027'},
 {'id': 'pu0028'},
 {'id': 'pu0029'},
 {'id': 'pu0030'},
 {'id': 'pu0031'},
 {'id': 'pu0032'},
 {'id': 'pu0033'},
 {'id': 'pu0034'},
 {'id': 'pu0035'},
 {'id': 'pu0036'},
 {'id': 'pu0037'},
 {'id': 'pu0038'},
 {'id': 'pu0039'},
 {'id': 'pu0040'},
 {'id': 'pu0041'},
 {'id': 'pu0042'},
 {'id': 'pu0043'},
 {'id': 'pu0044'},
 {'id': 'pu0045'},
 {'id': 'pu0046'},
 {'id': 'pu0047'},
 {'id': 'pu0048'},
 {'id': 'pu0049'},
 {'id': 'pu0050'},
 {'id': 'pu0051'},
 {'id': 'pu005

In [50]:
import nltk
from urllib import request
import requests

In [73]:
url = "https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Polish_wordlist"
html = requests.get(url)
page_content = BeautifulSoup(html.content, 'html.parser')

polishWords = []
for i in range(0, 5000):
    words = page_content.find_all("li")[i].text
    polishWords.append(words.split(' ', 1)[0])
    
polishWords[0:30]

['nie',
 'to',
 'się',
 'na',
 'co',
 'że',
 'jest',
 'do',
 'tak',
 'jak',
 'mnie',
 'ale',
 'mi',
 'za',
 'ja',
 'ci',
 'tu',
 'go',
 'tego',
 'tym',
 'ty',
 'czy',
 'tylko',
 'po',
 'jestem',
 'cię',
 'ma',
 'może',
 'już',
 'mam']